### Setup

In [21]:
from google.cloud import pubsub_v1
import json
import requests

### Publiser

In [17]:
topic_path = 'projects/auditpulse/topics/deployment-request-queue'
data = {
         'run_id': 81102,
         'user_id':1023,
         'central_index_key': '1249',
         'company_name':"Some Inc.",
         'ticker':'SINC',
         'year':2025
        }

jsonstring = json.dumps(data)
jsonstring = jsonstring.encode('utf-8')
publisher = pubsub_v1.PublisherClient()
future = publisher.publish(topic_path, jsonstring)

### Subscriber

In [18]:
def sub_callback(message):
    print(f'Data Received: {message}')
    global msg
    msg = message
    message.ack()
msg = None
subscriber_path = 'projects/auditpulse/subscriptions/deployment-request-queue-sub'
subscriber = pubsub_v1.SubscriberClient()
subscriber_future = subscriber.subscribe(subscriber_path, callback=sub_callback)
result = None
with subscriber:
    try:
        result = subscriber_future.result(timeout=5)
    except:
        subscriber_future.cancel()
        result = subscriber_future.result()

Data Received: Message {
  data: b'{"run_id": 81102, "user_id": 1023, "central_index_...'
  ordering_key: ''
  attributes: {}
}


In [19]:
msg

Message {
  data: b'{"run_id": 81102, "user_id": 1023, "central_index_...'
  ordering_key: ''
  attributes: {}
}

In [20]:
json.loads(msg.data)

{'run_id': 81102,
 'user_id': 1023,
 'central_index_key': '1249',
 'company_name': 'Some Inc.',
 'ticker': 'SINC',
 'year': 2025}

### Request

In [24]:
data = {
         'run_id': 81102,
         'user_id':1023,
         'central_index_key': '1249',
         'company_name':"Some Inc.",
         'ticker':'SINC',
         'year':2025
        }

jsonstring = json.dumps(data)
# jsonstring = jsonstring.encode('utf-8')

In [29]:
endpoint = 'http://127.0.0.1:5000/generate'
response = requests.post(endpoint, json={'message':{'data':jsonstring}})

In [30]:
response.text

'{\n  "message": "Report generated!",\n  "status": "Success!"\n}\n'